In [33]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [37]:
df = pd.read_csv("preprocessed_dataset.csv", parse_dates=["timestamp"])
df.head()

,requests,memory,cpu,timestamp
0,6.0,0.105,0.003,2023-10-01 00:00:00
1,29.0,0.104,0.004,2023-10-01 00:01:00
2,12.0,0.105,0.003,2023-10-01 00:02:00
3,18.0,0.105,0.004,2023-10-01 00:03:00
4,4.0,0.105,0.003,2023-10-01 00:04:00


In [38]:
df = pd.read_csv("preprocessed_dataset.csv", parse_dates=["timestamp"])

# Ensure data is sorted by timestamp
df = df.sort_values(by="timestamp").reset_index(drop=True)

# Select relevant columns
target_col = "requests"
feature_cols = ["memory", "cpu"]

# Normalize features
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

df[feature_cols] = scaler_x.fit_transform(df[feature_cols])
df[target_col] = scaler_y.fit_transform(df[[target_col]])

# Convert DataFrame to NumPy array
data = df[["requests", "memory", "cpu"]].values


In [43]:
data.shape

(1440, 3)

In [39]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_len=12, pred_len=12):  # Increased input_len to avoid lag errors
        self.data = data
        self.input_len = input_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.data) - self.input_len - self.pred_len

    def __getitem__(self, index):
        past_values = self.data[index : index + self.input_len]
        future_values = self.data[index + self.input_len : index + self.input_len + self.pred_len]
        
        # Create a binary mask (1 for observed values)
        past_observed_mask = np.ones_like(past_values)

        # Extract time features (e.g., normalized time step indices)
        past_time_features = np.expand_dims(np.arange(self.input_len) / self.input_len, axis=1)

        return (
            torch.tensor(past_values, dtype=torch.float32),
            torch.tensor(past_time_features, dtype=torch.float32),
            torch.tensor(past_observed_mask, dtype=torch.float32),
            torch.tensor(future_values, dtype=torch.float32),
        )

In [40]:
# Define sequence lengths
input_len = 24  # Increased to avoid lag-related errors
pred_len = 12   # Forecasting 12 time steps ahead


# Create dataset
dataset = TimeSeriesDataset(data, input_len, pred_len)
# Create DataLoader
train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)

In [41]:
len(dataset)

1404

In [ ]:
for batch in train_loader:
    print(f"Batch length: {len(batch)}")  # Should print 4
    print(f"past_values shape: {batch[0].shape}")  # (batch_size, input_len, features)
    print(f"past_time_features shape: {batch[1].shape}")  
    print(f"past_observed_mask shape: {batch[2].shape}")  
    print(f"future_values shape: {batch[3].shape}")  
    break  # Only print once


Batch length: 4
past_values shape: torch.Size([16, 112, 3])
past_time_features shape: torch.Size([16, 112, 1])
past_observed_mask shape: torch.Size([16, 112, 3])
future_values shape: torch.Size([16, 12, 3])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoformerConfig, AutoformerModel

# Define model configuration
config = AutoformerConfig(
    prediction_length=pred_len,
    context_length=input_len,  
    input_size=len(feature_cols),  
    lags_sequence=[1, 2, 3, 4],  # Manually define lags (must be <= context_length)
)

# Initialize model
model = AutoformerModel(config).to(device)


In [ ]:
model.parameters()

<generator object Module.parameters at 0x788f9c257ae0>

In [ ]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for past_values, past_time_features, past_observed_mask, future_values in train_loader:
        past_time_features, past_observed_mask, past_values = (
            past_time_features.to(device),
            past_observed_mask.to(device),
            past_values.to(device),
        )

        # print(past_time_features.shape)
        print("mask: ", past_observed_mask.shape)
        optimizer.zero_grad()
        
        # Forward pass (passing correct inputs)
        outputs = model(
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask,
            past_values=past_values
        ).last_hidden_state  # Extract predictions

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")


mask:  torch.Size([16, 112, 3])


ValueError: lags cannot go further than history length, found lag 4 while history length is only 112

### Demo

In [21]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoformerConfig, AutoformerForPrediction, Trainer, TrainingArguments

In [22]:
df = pd.read_csv("preprocessed_dataset.csv")

df['timestamp'] = pd.to_datetime(df['timestamp'])

input_features = ['memory', 'cpu']
target_feature = 'requests'

In [23]:
scaler = MinMaxScaler()
df[input_features + [target_feature]] = scaler.fit_transform(df[input_features + [target_feature]])

input_length = 61  # Past 48 time steps for input
prediction_length = 24 

In [ ]:
# Create dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, df, input_length, prediction_length):
        self.data = df
        self.input_length = input_length
        self.prediction_length = prediction_length
        
    def __len__(self):
        return len(self.data) - self.input_length - self.prediction_length + 1
    
    def __getitem__(self, idx):
        past_values = self.data.loc[idx : idx + self.input_length, target_feature].values
        future_values = self.data.loc[idx + self.input_length : idx + self.input_length + self.prediction_length, target_feature].values
        past_time_features = self.data.loc[idx : idx + self.input_length, input_features].values.reshape(self.input_length, -1)
        future_time_features = self.data.loc[idx + self.input_length : idx + self.input_length + self.prediction_length, input_features].values.reshape(self.prediction_length, -1)
        past_observed_mask = np.ones_like(past_values)  # Assume all values are observed
        future_observed_mask = np.ones_like(future_values)  # Ensure correct shape
        static_real_features = np.zeros((1, 1))  # Ensure shape is (1,)
        static_categorical_features = np.zeros((1, 1))  # Added static categorical features

        return {
            "past_values": torch.tensor(past_values, dtype=torch.float32).unsqueeze(-1),  # Shape (61, 1)
            "future_values": torch.tensor(future_values, dtype=torch.float32).unsqueeze(-1),  # Shape (24, 1)
            "past_time_features": torch.tensor(past_time_features, dtype=torch.float32).unsqueeze(1),  # Shape (61, 2)
            "future_time_features": torch.tensor(future_time_features, dtype=torch.float32).unsqueeze(1),  # Shape (24, 2)
            "past_observed_mask": torch.tensor(past_observed_mask, dtype=torch.float32).unsqueeze(-1),  # Shape (61, 1)
            "future_observed_mask": torch.tensor(future_observed_mask, dtype=torch.float32).unsqueeze(-1),  # Shape (24, 1)
            "static_real_features": torch.tensor(static_real_features, dtype=torch.float32).expand(1, 1),  # Shape (1, 1)
            "static_categorical_features": torch.tensor(static_categorical_features, dtype=torch.float32).expand(1, 1),  # Shape (1, 1)
        }

In [29]:
# Create dataset and dataloader
dataset = TimeSeriesDataset(df, input_length, prediction_length)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [26]:
sample = dataset[0]  # Fetch first sample
for key, value in sorted(sample.items()):
    print(f"{key}  {value.shape}")


future_observed_mask  torch.Size([25, 1])
future_time_features  torch.Size([25, 1, 2])
future_values  torch.Size([25, 1])
past_observed_mask  torch.Size([62, 1])
past_time_features  torch.Size([62, 1, 2])
past_values  torch.Size([62, 1])
static_categorical_features  torch.Size([1, 1])
static_real_features  torch.Size([1, 1])


In [30]:
# Load Autoformer model
config = AutoformerConfig(
    context_length=input_length,
    prediction_length=prediction_length,
    input_size=len(input_features),
    target_size=1
)
model = AutoformerForPrediction(config)

In [31]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=64,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [32]:
trainer.train()


RuntimeError: Tensors must have same number of dimensions: got 3 and 2